# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Grupo: Bruno Cury e Vitor Pereira<br /><br />
GitHub: 
https://github.com/Brunoic/Projeto2_Ciencia_dos_Dados

Arquivo excel no git com nome de "Kit Kat.xlsx"
* Conta: ***@CuryIzidoro***





___

## Preparando o ambiente

Instalando a biblioteca *tweepy* para realizar a conexão com o Twitter:

In [13]:
%%capture

#Instalando o tweepy
!pip install tweepy

Importando as Bibliotecas que serão utilizadas. Esteja livre para adicionar outras.

In [14]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter



In [22]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Coletando Dados


In [23]:
#Produto escolhido:
produto = 'Kit Kat'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [17]:
#Cria um objeto para a captura
#api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
#i = 1
#msgs = []
#for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
 #   msgs.append(msg.text.lower())
  #  i += 1
   # if i > n:
    #    break

#Embaralhando as mensagens para reduzir um possível viés
#shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [18]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

In [19]:
treinamento = pd.read_excel('Kit Kat.xlsx')
teste = pd.read_excel('Kit Kat.xlsx', sheetname = 'Teste')

#listas
Ltreinamento = []
Lteste = []
classificacao_Teste = []
classificacao_Treinamento = []
tweet_relevante = []
tweet_irrelevante = []
tipo_de_amostra = 1
quantidade_de_palavras = [0,0,0]

lista_freq_de_palavras= [["Kit Kat",0,0]]

#percorre o dataframe e o transforma em lista

for i in range (0,t):
    Ltreinamento.append([treinamento["Treinamento"][i],treinamento["Classificacao"][i]])
    
for i in range (0,t):
    classificacao_Treinamento.append(treinamento["Classificacao"][i])
    
for i in range (0,(n-t)):
    Lteste.append([teste["Teste"][i],teste["Classificacao"][i]])

for i in range (0,(n-t)):
    classificacao_Teste.append(teste["Classificacao"][i])


___
## Classificando as Mensagens

Capturamos mensagens do twitter sobre a marca KitKat e as classificamos em dois grupos, relevantes e irrelevantes

___
## Montando o Classificador Naive-Bayes


In [20]:
def multiply(n):
    total = 1
    for i in range(0, len(n)):
        total *= n[i]
    return total

class Classificador:
    
    def __init__(self,Ltreinamento,Lteste):
        
        self.Ltreinamento = Ltreinamento
        self.Lteste = Lteste
        self.lista_freq_de_palavras = lista_freq_de_palavras
        self.tweet_relevante = tweet_relevante
        self.tweet_irrelevante = tweet_irrelevante
        self.tipo_de_amostra = tipo_de_amostra
        self.quantidade_de_palavras = quantidade_de_palavras
        
    def normalize(self,Num):
        if (Num == 0): 
            lista = self.Ltreinamento
        else:
            lista = self.Lteste
        for tweet in lista:
            tweet[0] = tweet[0].replace(","," ")
            tweet[0] = tweet[0].replace("$"," ")
            tweet[0] = tweet[0].replace("+"," ")
            tweet[0] = tweet[0].replace("-"," ")
            tweet[0] = tweet[0].replace("="," ")
            tweet[0] = tweet[0].replace("%"," ")
            tweet[0] = tweet[0].replace("*"," ")
            tweet[0] = tweet[0].replace("/"," ")
            tweet[0] = tweet[0].replace("/n"," ")
            tweet[0] = tweet[0].replace(":"," ")
            tweet[0] = tweet[0].replace("("," ")
            tweet[0] = tweet[0].replace(")"," ")
            tweet[0] = tweet[0].replace("~"," ")
            tweet[0] = tweet[0].replace("["," ")
            tweet[0] = tweet[0].replace("]"," ")
            tweet[0] = tweet[0].replace("."," ")
            tweet[0] = tweet[0].replace(";"," ")
            tweet[0] = tweet[0].replace("_"," ")
            tweet[0] = tweet[0].replace("  "," ")
        if (Num == 0): 
            self.Ltreinamento = lista
        else:
            self.Lteste =  lista
            
            
    def treino(self):
        tweet_relevante = self.tweet_relevante
        tweet_irrelevante = self.tweet_irrelevante
        lista = self.Ltreinamento
        for tweet in lista:
            if tweet[1] == 1:
                tweet_relevante.append(tweet)
            else:
                tweet_irrelevante.append(tweet)    
            tweet[0] = tweet[0].split(' ')
            for palavra in tweet[0]:
                esta_na_lista = False
                for freq_de_palavras in self.lista_freq_de_palavras:
                    if freq_de_palavras[0] == palavra:
                        esta_na_lista = True
                        FR = freq_de_palavras[1]
                        FIR = freq_de_palavras[2]
                        if tweet in tweet_relevante:
                            FR += 1
                        else:
                            FIR += 1
                        freq_de_palavras[1] = FR
                        freq_de_palavras[2] = FIR
                        
                if not esta_na_lista:
                    tipo_de_amostra
                    if tweet in tweet_relevante:
                        self.lista_freq_de_palavras.append([palavra,1,0])
                    else:
                        self.lista_freq_de_palavras.append([palavra,0,1])
                            
    def analise(self,tweet): 
        PA =  len(self.tweet_relevante)/(300) #variando de 0 a 1
        PB =  len(self.tweet_irrelevante)/(300) #variando de 0 a 1
        tweet = tweet.split(' ')
        lista_Total = []
        lista_h = []
        lista_PhA = []
        lista_PhB = []
        for palavra in tweet:
            esta_na_lista = False
            for freq_de_palavras in self.lista_freq_de_palavras:
                if freq_de_palavras[0] == palavra:
                    esta_na_lista = True
                    Ph = (freq_de_palavras[1]+freq_de_palavras[2]+1)/self.tipo_de_amostra
                    PhA = (freq_de_palavras[1]+1)/(self.tipo_de_amostra + self.quantidade_de_palavras[1])
                    PhB = (freq_de_palavras[1]+1)/(self.tipo_de_amostra + self.quantidade_de_palavras[2])
                    lista_h.append(Ph)
                    lista_PhA.append(PhA)
                    lista_PhB.append(PhB)
                    lista_Total.append([Ph,PhB,PhA])
            
            if not esta_na_lista:
                lista_h.append(1/tipo_de_amostra)
                lista_PhA.append(1/self.tipo_de_amostra + self.quantidade_de_palavras[1])
                lista_PhB.append(1/self.tipo_de_amostra + self.quantidade_de_palavras[2])
        
        PH = multiply(lista_h)
        PHrel = multiply(lista_PhA)
        PHirel = multiply(lista_PhB)
        bayes_relevante = (PHrel * PA)/PH
        bayes_irrelevante = (PHirel * PB)/PH
        if (bayes_relevante>bayes_irrelevante):
            return 1
        else:
            return 0
        
    def teste(self):
        sim_acerto = 0
        sim_erro = 0
        nao_acerto = 0
        nao_erro = 0
        total_acerto = 0
        total_erro = 0
        for tweet in self.Lteste:
            analise =self.analise(tweet[0])
            if analise == tweet[1]:
                total_acerto += 1
                if tweet[1]==1:sim_acerto += 1
                else:nao_acerto += 1
            else:
                total_erro += 1
                if analise==1:
                    sim_erro += 1
                else:nao_erro += 1
        print("total de erros: " +str(total_erro))
        print("total de acertos " +str(total_acerto))
        print()
        print("positivo falso " +str(sim_erro))
        print("positivo verdadeiro " +str(sim_acerto))
        print()
        print("negativo falso " +str(nao_erro))
        print("negativo verdadeiro " +str(nao_acerto))

___
## Verificando a performance



In [21]:
Classifica = Classificador(Ltreinamento,Lteste)
Classifica.normalize(0)
Classifica.normalize(1)
Classifica.treino()
Classifica.teste()

total de erros: 50
total de acertos 150

positivo falso 0
positivo verdadeiro 0

negativo falso 50
negativo verdadeiro 150


    ___
    
## Concluindo

   Criamos um classificador Naive-Bayes que analisa a probabilidade de um tweet sobre a marca KitKat ser relevante, baseando-se nas palavras que são utilizadas no tweet. Para tanto fizemos uma classificação manual de alguns tweets que serviram de base do nosso banco de dados para a classificação dos tweets e posteriormente serviram como comparação para avaliar os resultados obtidos pelo classificador Naive-Bayes.
    
   Como nosso classificador trabalha com probabilidades, ele não tem a habilidade de julgar termos que são utilizados com tom de ironía, dupla negação ou sarcasmo, e pelo fato da rede social "Twitter" ser um ambiente informal, há um grande número de comentários que apresentam essas características que nosso classificador Naive-Bayes não consegue diferir os sentidos. Esse fator faz com que o classificador não seja tao preciso quanto desejado, e tenhamos nos resultados um número de erros maior do que o esperado.

   Uma ideia interessante para aprimorar os resultados é dar ao classificador a habilidade de julgar termos que são utilizados com intenção de expressar ironia, dupla negação e sarcasmo, além disso outra ideia seria aumentar o numero de tweets manualmente classificados para que forneça ao classificador uma maior variação de palavras já classificadas, resultando numa diminuição dos erros.

   Não seria uma decisão inteligente realimentar o nosso classificador com tweets por ele mesmo classificado pois como a classificação é baseada na área de treinamento, tweets mal classificados acarretariam um aumento na quantidade de erros cometidos pelo classificador.
    
   O sistema que usamos, classificador Naive-Bayes, é confiavel e por isso muito utilizado atualmente principalmente em pesquisas de mercado para saber qual a visão do consumidor sobre a marca, as ações por ela tomadas e como eles reagem ao lançamento de novos produtos, além disso críticas de filmes, serviços prestados a sociedade, etc. Essa ferramenta pode ser utilizada também numa tentativa de "prever o futuro", com previsões do tempo, mercado de apostas, mercado de ações, entre outros.


